In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
%run train.py --clearml-run-name beta_1 --batch-size 128 --test-batch-size 256 --epochs 201 --lr 0.001 --beta 1.0 --model-name beta_1